# All the files can be found in the given link 
Please use the link and download the full folder and use the maincodefile.ipynb to test the solution

link: https://drive.google.com/drive/folders/1y0FeqhW9LKK624gKfodhI0V8GgOdv6SM?usp=sharing

# Importing the Libraries and the machine learning code

In [1]:
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QLabel, QRadioButton, QLineEdit, QPushButton, QVBoxLayout, QHBoxLayout, QFileDialog
from PyQt5.QtGui import QPixmap
from PyQt5.QtCore import Qt
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import mltest as md


c:\Users\Admin\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.1.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\Admin\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.1.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\Admin\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator StandardScale

# Image Processing code to process the test image

In [2]:
def imgprocess(path, name):
    # dataStoredAt = '{}\{}'
    # dataStoredAt = dataStoredAt.format(path[:-4],'diseased.csv')  
    # fpath = '{}\{}'
    # fpath = fpath.format(mypath,iname)
    img = cv2.imread(path)
    lower = np.array([0,70,52])
    upper = np.array([163,255,255])
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower, upper)
    output = cv2.bitwise_and(img,img, mask= mask)
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        
        cv2.drawContours(img,[cnt],-1, (0, 0, 255), 3)
    # spath = '{}\{}'
    # spath = spath.format(path,iname)
    cv2.imwrite(name+'.jpg', img)
    conts = []
    for i in contours:
        if len(i)>200:
            print(len(i))
            conts.append(i)
    print(len(conts))
    if len(conts)>0:
        newconts = np.array_split(conts[0],30)
    
        length = 0
        alllengths = []
        allcont = []
        checkpoints = []
        for i in newconts:
            for j in range(len(i)):
                if j!= len(i)-1:
                    length+= math.sqrt(math.pow((i[j+1][0][0]-i[j][0][0]),2)+math.pow((i[j+1][0][1]-i[j][0][1]),2))
            displacement = math.sqrt(math.pow((i[-1][0][0]-i[0][0][0]),2)+math.pow((i[-1][0][1]-i[0][0][1]),2))
            checkpoints.append([i[-1][0][0],i[-1][0][1]])
            alllengths.append([length,displacement])
            
            length = 0
        # print(alllengths)
        # print(checkpoints)
        diff = []
        # f = open(dataStoredAt,'a')
        # f.write(str(iname)+',')
        for i in alllengths:
            dif = ((i[0]-i[1])/i[1])*100
            # f.write(str(dif)+',')
            diff.append(dif)
        # f.write('\n')
        # f.close()
        return diff

# GUI Code

In [3]:
class MainWindow(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        # Create name text box
        self.text_name = QLineEdit()

        # Create radio buttons
        self.radio_male = QRadioButton('Male')
        self.radio_female = QRadioButton('Female')
        self.radio_male.setChecked(True)  # Set default as Male

        # Create layouts for name and gender
        layout_name_gender = QHBoxLayout()
        layout_name_gender.addWidget(QLabel('Name: '))
        layout_name_gender.addWidget(self.text_name)
        layout_name_gender.addWidget(QLabel('Gender: '))
        layout_name_gender.addWidget(self.radio_male)
        layout_name_gender.addWidget(self.radio_female)

        # Create image box
        self.image_label = QLabel()
        self.image_label.setFixedSize(200, 200)

        # Create video box
        self.video_label = QLabel()
        self.video_label.setFixedSize(200, 200)

        # Create buttons
        self.btn_upload_video = QPushButton('Upload Image')
        self.btn_upload_video.clicked.connect(self.openVideo)
        self.btn_generate_report = QPushButton('Generate Report')
        self.btn_generate_report.clicked.connect(self.generateReport)

        # Create layouts for video and image
        layout_video_image = QHBoxLayout()
        layout_video_image.addWidget(self.video_label)
        layout_video_image.addWidget(self.image_label)

        # Create layout for buttons
        layout_buttons = QHBoxLayout()
        layout_buttons.addWidget(self.btn_upload_video)
        layout_buttons.addWidget(self.btn_generate_report)

        # Create main layout
        layout_main = QVBoxLayout()
        layout_main.addLayout(layout_name_gender)
        layout_main.addLayout(layout_video_image)
        layout_main.addLayout(layout_buttons)

        self.setLayout(layout_main)
        self.setWindowTitle('My Application')
        self.show()

        self.video_path = ""

    def openVideo(self):
        filename, _ = QFileDialog.getOpenFileName(self) #, "Select Video", "", "Video Files (*.mov)"
        if filename:
            # self.video_label.setText(filename)
            self.video_path = filename

    def generateReport(self):
        gender = 'Male' if self.radio_male.isChecked() else 'Female'
        name = self.text_name.text()
        if self.video_path:
            print(f"Name: {name}\nGender: {gender}\nVideo Path: {self.video_path}")
            pred = md.pred([imgprocess(self.video_path,name)])
            if pred == '0':
                prediction  = 'Negative'
            else:
                prediction = 'Positive'
            # print(imgprocess(self.video_path,name))
            print(prediction, md.pred([imgprocess(self.video_path,name)]))
            pixmap = QPixmap(name+'.jpg')
            pixmap = pixmap.scaled(200, 200, Qt.KeepAspectRatio)
            self.image_label.setPixmap(pixmap)
            # imgprocess(self.video_path)
            
            self.video_label.setText('Name:'+name+' Gender:'+gender+'\nprediction result:'+prediction)
            # report(self.video_path)
        else:
            print("Please select a video file.")

In [ ]:
if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = MainWindow()
    sys.exit(app.exec_())